In [1]:
from io import BytesIO
from urllib import request
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow.lite as tflite

2023-12-16 19:31:24.033451: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-12-16 19:31:24.033547: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
path = '../data/test/baseball/4.jpg'

In [3]:
user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
headers={'User-Agent':user_agent,} 

def download_image(url):
    respose = request.Request(url,None,headers)
    with request.urlopen(respose) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img

In [4]:
def load_preprocess_img(img_path):
    if img_path.startswith('https') or img_path.startswith('http'):
        img = download_image(img_path)
    else:
        # Load the image
        img = Image.open(img_path)

    img = img.resize((224,224))

    # Convert to numpy array
    x = np.array(img)
    # Convert x into a batch
    X = np.array([x], dtype='float32')
    # Preprocess Image
    X /= 255.0
    return X

In [6]:
url = 'https://images.pexels.com/photos/3628912/pexels-photo-3628912.jpeg?auto=compress&cs=tinysrgb&w=1260&h=750&dpr=1'

In [7]:
X = load_preprocess_img(url)
X.shape

(1, 224, 224, 3)

### Using TFlite

In [8]:
interpreter = tflite.Interpreter(model_path='../models/prediction.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

In [9]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)

In [10]:
classes = ['airhockey',
'amputefootball',
'archery',
'armwrestling',
'axethrowing',
'balancebeam',
'barellracing',
'baseball',
'basketball',
'batontwirling',
'bikepolo',
'billiards',
'bmx',
'bobsled',
'bowling',
'boxing',
'bullriding',
'bungeejumping',
'canoeslamon',
'cheerleading',
'chuckwagonracing',
'cricket',
'croquet',
'curling',
'discgolf',
'fencing',
'fieldhockey',
'figureskatingmen',
'figureskatingpairs',
'figureskatingwomen',
'flyfishing',
'football',
'formularacing',
'frisbee',
'gaga',
'giantslalom',
'golf',
'hammerthrow',
'hanggliding',
'harnessracing',
'highjump',
'hockey',
'horsejumping',
'horseracing',
'horseshoepitching',
'hurdles',
'hydroplaneracing',
'iceclimbing',
'iceyachting',
'jaialai',
'javelin',
'jousting',
'judo',
'lacrosse',
'logrolling',
'luge',
'motorcycleracing',
'mushing',
'nascarracing',
'olympicwrestling',
'parallelbar',
'poleclimbing',
'poledancing',
'polevault',
'polo',
'pommelhorse',
'rings',
'rockclimbing',
'rollerderby',
'rollerbladeracing',
'rowing',
'rugby',
'sailboatracing',
'shotput',
'shuffleboard',
'sidecarracing',
'skijumping',
'skysurfing',
'skydiving',
'snowboarding',
'snowmobileracing',
'speedskating',
'steerwrestling',
'sumowrestling',
'surfing',
'swimming',
'tabletennis',
'tennis',
'trackbicycle',
'trapeze',
'tugofwar',
'ultimate',
'unevenbars',
'volleyball',
'watercycling',
'waterpolo',
'weightlifting',
'wheelchairbasketball',
'wheelchairracing',
'wingsuitflying']

In [11]:
sorted(zip(classes, preds.squeeze()), key=lambda x:-x[1])[:5]

[('cricket', 10.8796835),
 ('baseball', 8.193538),
 ('croquet', 7.908885),
 ('football', 6.1987066),
 ('golf', 5.5249014)]